This script pulls address data from a csv, and uses the google maps api to get the corresponding gps coordinates and neighborhood, then outputs a new csv


In [3]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import pprint

In [4]:
#pull data from csv into pandas datafrom 
data = pd.read_csv("murals.csv")

In [6]:
#remove unnecessary column
data = data.drop("Unnamed: 0", axis = 1)

In [7]:
#first 5 rows
data.head()

,muralName,artistName,address,year,imgLink
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...


In [11]:
#look at address field 
data.loc[1].address

'19031 Grand River Ave, Detroit, MI'

In [15]:
#call api from above address

api_key = "" #removed api key
address = data.loc[1].address
address = address.replace(' ', '+')
api_call = 'https://maps.googleapis.com/maps/api/geocode/json?address='+address+'&key='+api_key


In [26]:
#check out json results from api
import json
response = requests.get(api_call)
print(response.content.decode("utf-8"))

{
   "results" : [
      {
         "address_components" : [
            {
               "long_name" : "19031",
               "short_name" : "19031",
               "types" : [ "street_number" ]
            },
            {
               "long_name" : "Grand River Avenue",
               "short_name" : "Grand River Ave",
               "types" : [ "route" ]
            },
            {
               "long_name" : "Rosedale Park",
               "short_name" : "Rosedale Park",
               "types" : [ "neighborhood", "political" ]
            },
            {
               "long_name" : "Detroit",
               "short_name" : "Detroit",
               "types" : [ "locality", "political" ]
            },
            {
               "long_name" : "Wayne County",
               "short_name" : "Wayne County",
               "types" : [ "administrative_area_level_2", "political" ]
            },
            {
               "long_name" : "Michigan",
               "short_name" : "MI

In [62]:
#removes rows with null values in any column
data = data.dropna().reset_index(drop=True)


In [82]:
#remove row 95
data = data.drop(data.index[95])

In [84]:
#resets index (needed to loop through the dataframe)
data = data.reset_index(drop=True)

In [47]:
#check for gps coordinates in json
print(response.json().get('results')[0].get('geometry').get('location').get('lat'))
response.json().get('results')[0].get('geometry').get('location').get('lng')

42.40459300000001


-83.228594

In [8]:
api_key = ""


In [114]:
#create list to store lat and long data
lat = []
long = []
for i in range(len(data)):
    #clean up addresses
    address = data.loc[i].address.replace("(near) ","")
    address = address.replace("(In Alley)","")
    address = address.replace("(in Alley)","")
    address = address.replace("(Alley behind building)","")
    address = address.replace( "(parking lot)","")
    address = address.replace("(alley)","")
    address = address.replace("(Rear)","")
    
    print(address)
    address = address.replace(' ', '+')
    #call api
    api_call = 'https://maps.googleapis.com/maps/api/geocode/json?address='+address+'&key='+api_key
    
    response = requests.get(api_call)
    
    #add coordinates to list
    lat.append(response.json().get('results')[0].get('geometry').get('location').get('lat'))
    long.append(response.json().get('results')[0].get('geometry').get('location').get('lng'))
    
                              
    

2605 Newark Street, Detroit, MI
19031 Grand River Ave, Detroit, MI
2605 Newark Street, Detroit, MI
8801 Oakland Avenue, Detroit, MI
4126 Third Avenue, Detroit, MI
7900 Mack Avenue, Detroit, MI
10125 East Jefferson, Detroit, MI
6600 West Vernor Highway, Detroit, MI
1550 Taylor Street, Detroit, MI
4847 Fort Street, Detroit, MI
4847 Fort Street, Detroit, MI
3546 Michigan Avenue, Detroit, MI
1457 Griswold St, Detroit, MI
431 E Congress St, Detroit, MI
6060 John R Street, Detroit, MI
4475 2nd Avenue, Detroit, MI
8359 Woodward Ave, Detroit, MI
8359 Woodward Avenue, Detroit, MI
3714 Trumbull Street, Detroit, MI
1431 Washington Blvd, Detroit, MI
314 East Baltimore Ave, Detroit MI
124 West Columbia Street, Detroit, MI
1527 West Warren Ave, Detroit, MI
1922 Ferdinand Street, Detroit, MI
9429 West Fort Street, Detroit, MI
7830 W Vernor Hwy, Detroit, Michigan
12355 Gratiot Ave., Detroit
1930 Springwells Ave, Detroit, MI
5432 Michigan Avenue, Detroit, MI
2917 Gratiot, Detroit, MI
1444 Brooklyn St D

In [115]:
#look at last 5 rows in dataframe
data.tail()

,muralName,artistName,address,year,imgLink
163,DETROIT MURAL #0112,Ron Zakrin,"1516 Winder St (In Alley), Detroit, MI",2015,https://static1.squarespace.com/static/5ab0771...
164,DETROIT MURAL #0113,2501,"1550 Winder Street, Detroit, MI",2015,https://static1.squarespace.com/static/5ab0771...
165,DETROIT MURAL #0264,Beau Stanton,"2126 Pierce St, Detroit, MI",2015,https://static1.squarespace.com/static/5ab0771...
166,DETROIT MURAL #0372,Sintex,"2144 Gratiot Ave., Detroit, MI",2014,https://static1.squarespace.com/static/5ab0771...
167,DETROIT MURAL #0371,Sintex,"2144 Gratiot Ave., Detroit, MI",2014,https://static1.squarespace.com/static/5ab0771...


In [116]:
#add lat and long to dataframe
data['lat']=lat
data['lng'] = long

In [117]:
#show dataframe now with coordinates
data

,muralName,artistName,address,year,imgLink,lat,lng
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.327435,-83.079290
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.404593,-83.228594
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.327435,-83.079290
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.383743,-83.071422
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.348580,-83.067387
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.366416,-83.006741
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.362034,-82.981698
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.316220,-83.114001
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.376944,-83.095842
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.310107,-83.094345


In [130]:
#look at json for neighborhood
response.json().get("results")[0].get("address_components")[2].get('short_name')

'Lafayette Park'

In [133]:
#create list to store neighborhood info and populate it
neighborhood=[]
for i in range(len(data)):
    address = data.loc[i].address.replace("(near) ","")
    address = address.replace("(In Alley)","")
    address = address.replace("(in Alley)","")
    address = address.replace("(Alley behind building)","")
    address = address.replace( "(parking lot)","")
    address = address.replace("(alley)","")
    address = address.replace("(Rear)","")
    
    print(address)
    address = address.replace(' ', '+')
    api_call = 'https://maps.googleapis.com/maps/api/geocode/json?address='+address+'&key='+api_key
    #print(api_call)
    response = requests.get(api_call)
    hood = response.json().get("results")[0].get("address_components")[2].get('short_name')
    print(hood)
    neighborhood.append(hood)

2605 Newark Street, Detroit, MI
Hubbard - Richard
19031 Grand River Ave, Detroit, MI
Rosedale Park
2605 Newark Street, Detroit, MI
Hubbard - Richard
8801 Oakland Avenue, Detroit, MI
North End
4126 Third Avenue, Detroit, MI
Midtown
7900 Mack Avenue, Detroit, MI
Islandview
10125 East Jefferson, Detroit, MI
Detroit
6600 West Vernor Highway, Detroit, MI
Detroit
1550 Taylor Street, Detroit, MI
Virginia Park
4847 Fort Street, Detroit, MI
Delray
4847 Fort Street, Detroit, MI
Delray
3546 Michigan Avenue, Detroit, MI
Mexicantown - Southwest Detroit
1457 Griswold St, Detroit, MI
Downtown Detroit
431 E Congress St, Detroit, MI
Downtown Detroit
6060 John R Street, Detroit, MI
Milwaukee Junction
4475 2nd Avenue, Detroit, MI
Midtown
8359 Woodward Ave, Detroit, MI
Virginia Park
8359 Woodward Avenue, Detroit, MI
Virginia Park
3714 Trumbull Street, Detroit, MI
Woodbridge
1431 Washington Blvd, Detroit, MI
Downtown Detroit
314 East Baltimore Ave, Detroit MI
Milwaukee Junction
124 West Columbia Street, De

In [134]:
#check length of list, does it match length of dataframe?
len(neighborhood)

168

In [135]:
#add neighborhood info to dataframe
data['neighborhood'] = neighborhood

In [136]:
data

,muralName,artistName,address,year,imgLink,lat,lng,neighborhood
0,DETROIT MURAL #0674,Pat Perry,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.327435,-83.079290,Hubbard - Richard
1,DETROIT MURAL #0694,Tony Whlgn,"19031 Grand River Ave, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.404593,-83.228594,Rosedale Park
2,DETROIT MURAL #0693,Ledania,"2605 Newark Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.327435,-83.079290,Hubbard - Richard
3,DETROIT MURAL #0690,FFTY,"8801 Oakland Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.383743,-83.071422,North End
4,DETROIT MURAL #0689,Laura Finlay,"4126 Third Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.348580,-83.067387,Midtown
5,DETROIT MURAL #0687,Ndubisi Okoye,"7900 Mack Avenue, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.366416,-83.006741,Islandview
6,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,"10125 East Jefferson, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.362034,-82.981698,Detroit
7,DETROIT MURAL #0671,Bernal Perez,"6600 West Vernor Highway, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.316220,-83.114001,Detroit
8,DETROIT MURAL #0682,Tylonn J. Sawyer,"1550 Taylor Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.376944,-83.095842,Virginia Park
9,DETROIT MURAL #0625,Patch Whisky,"4847 Fort Street, Detroit, MI",2018,https://static1.squarespace.com/static/5ab0771...,42.310107,-83.094345,Delray


In [137]:
#pull other csv (hardcoded data) to new dataframe
data1= pd.read_csv("mural1.csv")

In [138]:
data1

,imgloc,latitude,longitude,address,neighborhood,tags,muralid,name,artistname,favoritecount,info
0,https://i.imgur.com/mjJSRfI.jpg,42.335615,-83.049770,"1050 Woodward Avenue, Detroit MI",NaN,NaN,1,#WhatLiftsYou,Kelsey Montague,0,NaN
1,https://www.wyland.com/media/20110209104734%20...,42.336082,-83.049828,"1565 Broadway St, Detroit, MI",NaN,NaN,2,Whale Mural @ Broderick Towers,Wyland,0,NaN
2,http://patperry.net/cache/images/local/uploads...,42.327435,-83.079290,"2605 Newark Street, Detroit, MI",NaN,NaN,3,Pat Perry Mural,Pat Perry,0,NaN
3,https://www.modeldmedia.com/Images/Features/is...,42.315591,-83.113959,"6600 West Vernor Highway, Detroit, MI",NaN,NaN,4,Bernal Perez Raft,Bernal Perez,0,NaN
4,https://www.modeldmedia.com/Images/Features/is...,42.312246,-83.125335,"7830 Vernor Hwy, Detroit MI",NaN,NaN,5,Flagrant,Ellen Rutt,0,NaN
5,https://www.modeldmedia.com/Images/Features/is...,42.363101,-82.989453,"9300 Kercheval Ave, Detroit MI",NaN,NaN,6,WC Bevan- corner mural,W.C. Bevan,0,NaN
6,https://www.modeldmedia.com/Images/Features/is...,42.366416,-83.008930,7900 Mack Ave,NaN,NaN,7,Therefore go and make disciples of all nations,Ndubisi Okoye,0,NaN
7,https://www.modeldmedia.com/Images/Features/is...,42.295235,-83.133233,"9429 W Fort St, Detroit, MI",NaN,NaN,8,High Dro Zone,Freddy Diaz,0,NaN
8,https://www.modeldmedia.com/Images/Features/is...,42.442185,-83.166278,"20194 Mendota St, Detroit, MI",NaN,NaN,9,Birwood Wall,Various -Designed by Chazz Miller,0,http://www.detroiturbex.com/content/neighborho...
9,https://www.modeldmedia.com/Images/Features/is...,42.429149,-83.264153,"22298 W 7 Mile Rd, Detroit MI",NaN,NaN,10,Fractal by HOXXOH,HOXXOH,0,NaN


In [139]:
#add neighborhood data to new dataframe
neighborhood=[]
for i in range(len(data1)):
    address = data1.loc[i].address.replace("(near) ","")
    address = address.replace("(In Alley)","")
    address = address.replace("(in Alley)","")
    address = address.replace("(Alley behind building)","")
    address = address.replace( "(parking lot)","")
    address = address.replace("(alley)","")
    address = address.replace("(Rear)","")
    
    print(address)
    address = address.replace(' ', '+')
    api_call = 'https://maps.googleapis.com/maps/api/geocode/json?address='+address+'&key='+api_key
    #print(api_call)
    response = requests.get(api_call)
    hood = response.json().get("results")[0].get("address_components")[2].get('short_name')
    print(hood)
    neighborhood.append(hood)

1050 Woodward Avenue, Detroit MI
Downtown Detroit
1565 Broadway St, Detroit, MI
Downtown Detroit
2605 Newark Street, Detroit, MI
Hubbard - Richard
6600 West Vernor Highway, Detroit, MI
Detroit
7830 Vernor Hwy, Detroit MI
Springwells Village
 9300 Kercheval Ave, Detroit MI
English Village
7900 Mack Ave
Islandview
9429 W Fort St, Detroit, MI
Carbon Works
20194 Mendota St, Detroit, MI 
Eight Mile Wyoming
22298 W 7 Mile Rd, Detroit MI
Detroit
12355 Gratiot Ave
Detroit
4765 14th St
Deerfield Beach


In [140]:
data1['neighborhood'] = neighborhood

In [149]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 8 columns):
muralName       168 non-null object
artistName      168 non-null object
address         168 non-null object
year            168 non-null object
imgLink         168 non-null object
lat             168 non-null float64
lng             168 non-null float64
neighborhood    168 non-null object
dtypes: float64(2), object(6)
memory usage: 10.6+ KB


In [168]:
#add new dataframe to main dataframe
for i in range(len(data1)):
    data.loc[len(data)] = [data1.loc[i].name, data1.loc[i].artistname, data1.loc[i].address, "", data1.loc[i].imgloc, data1.loc[i].latitude, data1.loc[i].longitude, data1.loc[i].neighborhood]


In [175]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 8 columns):
muralName       180 non-null object
artistName      180 non-null object
address         180 non-null object
year            180 non-null object
imgLink         180 non-null object
lat             180 non-null float64
lng             180 non-null float64
neighborhood    180 non-null object
dtypes: float64(2), object(6)
memory usage: 12.7+ KB


In [145]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 8 columns):
muralName       168 non-null object
artistName      168 non-null object
address         168 non-null object
year            168 non-null object
imgLink         168 non-null object
lat             168 non-null float64
lng             168 non-null float64
neighborhood    168 non-null object
dtypes: float64(2), object(6)
memory usage: 10.6+ KB


In [188]:
cols = data.columns.tolist()

In [189]:
cols

['imgloc',
 'latitude',
 'longitude',
 'address',
 'neighborhood',
 'name',
 'artistname',
 'info']

In [203]:
#change columns to match pojo and database structure
columns = ['imgloc', 'latitude', 'longitude', 'address', 'neighborhood','tags', 'name','artistname', 'info']


In [204]:
data = data[columns]

In [205]:
data


,imgloc,latitude,longitude,address,neighborhood,tags,name,artistname,info
0,https://static1.squarespace.com/static/5ab0771...,42.327435,-83.079290,"2605 Newark Street, Detroit, MI",Hubbard - Richard,,DETROIT MURAL #0674,Pat Perry,2018
1,https://static1.squarespace.com/static/5ab0771...,42.404593,-83.228594,"19031 Grand River Ave, Detroit, MI",Rosedale Park,,DETROIT MURAL #0694,Tony Whlgn,2018
2,https://static1.squarespace.com/static/5ab0771...,42.327435,-83.079290,"2605 Newark Street, Detroit, MI",Hubbard - Richard,,DETROIT MURAL #0693,Ledania,2018
3,https://static1.squarespace.com/static/5ab0771...,42.383743,-83.071422,"8801 Oakland Avenue, Detroit, MI",North End,,DETROIT MURAL #0690,FFTY,2018
4,https://static1.squarespace.com/static/5ab0771...,42.348580,-83.067387,"4126 Third Avenue, Detroit, MI",Midtown,,DETROIT MURAL #0689,Laura Finlay,2018
5,https://static1.squarespace.com/static/5ab0771...,42.366416,-83.006741,"7900 Mack Avenue, Detroit, MI",Islandview,,DETROIT MURAL #0687,Ndubisi Okoye,2018
6,https://static1.squarespace.com/static/5ab0771...,42.362034,-82.981698,"10125 East Jefferson, Detroit, MI",Detroit,,DETROIT MURAL #0686,Michael Burdick and Kelly Golden,2018
7,https://static1.squarespace.com/static/5ab0771...,42.316220,-83.114001,"6600 West Vernor Highway, Detroit, MI",Detroit,,DETROIT MURAL #0671,Bernal Perez,2018
8,https://static1.squarespace.com/static/5ab0771...,42.376944,-83.095842,"1550 Taylor Street, Detroit, MI",Virginia Park,,DETROIT MURAL #0682,Tylonn J. Sawyer,2018
9,https://static1.squarespace.com/static/5ab0771...,42.310107,-83.094345,"4847 Fort Street, Detroit, MI",Delray,,DETROIT MURAL #0625,Patch Whisky,2018


In [211]:
data.tail()

,imgloc,latitude,longitude,address,neighborhood,tags,name,artistname,info
175,https://www.modeldmedia.com/Images/Features/is...,42.295235,-83.133233,"9429 W Fort St, Detroit, MI",Carbon Works,,7,Freddy Diaz,
176,https://www.modeldmedia.com/Images/Features/is...,42.442185,-83.166278,"20194 Mendota St, Detroit, MI",Eight Mile Wyoming,,8,Various -Designed by Chazz Miller,
177,https://www.modeldmedia.com/Images/Features/is...,42.429149,-83.264153,"22298 W 7 Mile Rd, Detroit MI",Detroit,,9,HOXXOH,
178,https://www.modeldmedia.com/Images/Features/is...,42.416791,-82.992306,12355 Gratiot Ave,Detroit,,10,Jonny Alexander,
179,https://www.modeldmedia.com/Images/Features/is...,42.348266,-83.088302,4765 14th St,Deerfield Beach,,11,Michael Olszewski,


In [207]:
#export to csv
data.to_csv("muralfinal.csv")